In [ ]:
!nvcc --version
!nvidia-smi
import os, shutil
import numpy as np
import matplotlib.pyplot as plt
from cellpose import core, utils, io, models, metrics, transforms
from tqdm import trange
import fastremap


use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')



In [ ]:
# call logger_setup to have output of cellpose written
from cellpose.io import logger_setup
from cellpose.transforms import normalize99
from cellpose.io import imsave

logger_setup();

path_input = "Path/To/DAPI"
filename = "DAPI.tif"

path = os.path.join(path_input, filename)

print(path)
dapi = io.imread(path)


print("dapi channel")
print("\r shape: {0}".format(dapi.shape))
print("\r dtype: {0}".format(dapi.dtype))
print("\r min: {0}".format(dapi.min()))
print("\r max: {0}".format(dapi.max()), "\n")



In [ ]:
dapi_norm = normalize99(dapi)
#plt.imshow(dapi_norm[round(dapi.shape[0]/2)], vmin=0, vmax=1)


In [ ]:

path_output = path_input + 'segmentation/';
os.makedirs(path_output, exist_ok=True)



In [ ]:
# model name and path
#@markdown ###Custom model path (full path):
model_path = "Path/To/Model" #@param {type:"string"}


# declare model
model = models.CellposeModel(gpu=True, 
                             pretrained_model=model_path)

# use model diameter if user diameter is 0
diameter = 0
diameter = model.diam_labels if diameter==0 else diameter
print(diameter)

In [ ]:
rfilename = filename.split('.')
rfilename.extend({'masks_1000pixelcutoff_trainedmodel'})
rfilename = '_'.join(rfilename) + '.tif'
rfilename = os.path.join(path_output,  rfilename)
print(rfilename)


In [ ]:
masks, flows, styles = model.eval(dapi_norm, 
                                    diameter=diameter,
                                    flow_threshold=0.0,
                                    do_3D=True, normalize=False)

In [ ]:
print(masks.shape)

In [ ]:
masks_final = utils.fill_holes_and_remove_small_masks(masks.copy(), min_size=1000)
print(f"removed {masks.max() - masks_final.max()} masks smaller than 1000 pixels")


In [ ]:
print(masks_final.max())

In [ ]:
io.imsave(rfilename, masks_final)